In [51]:
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns

jobs = pd.read_pickle(r'Notebooks/jeff-jobs-cleaned-2025.pkl')
supply = pd.read_pickle(r'Notebooks/jeff-supply-cleaned-2025.pkl')
#stages = pd.read_pickle(r'Notebooks/work_breakdown.pkl')

In [52]:
#Each of these categories is similar to 'Lack of facilities/capabilities'
to_group = [ "No formal training on this equipment","Not authorized for ship's force accomplishment","Inadequate school practical training",
       "Formal training inadequate for this equipment", "Lack of technical documentation"]
jobs['deferral_reason_reduced'] = jobs['deferral_reason'].apply(lambda x: 'Lack of facilities/capabilities' if x in to_group else x)

# Goal: Identify The Equipment System Being Worked On

# System Frequencies [~ 50 Manually Tagged as Early Trial, still ~ 10k records of unidentified entries]
<pre>
Ventilation System: 10
AC System: 18
MC System: 10
Heating System: 2
Chilled Water System: 5
Unknown: 3


In [53]:
equipment_map = { #35502 records accounted for
    'VENT NONMCH PRP': 'Ventilation System',
    'VENTILATION SYST': 'Ventilation System',
    'VENT MACH SPCS F': 'Ventilation System',
    'AIR CONDITIONING': 'AC System',
    'NON-MACHINERY SP': 'Unknown',
    'ANNOUNCING SYSTE': 'MC System',
    'SVCE STM POTW HT': 'Heating System',
    'IC CKT 1MC LOUDS': 'MC System',
    'VENTILATION ACCE' : 'Ventilation System',
    'IC CKT 1MC/3MC A': 'MC System',
    'PLENUM, VENTILAT': 'Ventilation System',
    'IC CKT 5MC LOUDS': 'MC System',
    'HEATING SYS CONV': 'Heating System',
    'IC CKT 1MC/3MC C': 'MC System',
    'VENT MACH SPCS S' : 'Ventilation System',
    'CHW SYS DRINKING': 'Chilled Water System',
    'CHW SYS DUCT TYP': 'Chilled Water System',
    'SEE CDM FOR INFO': 'Unknown',
    'IM-239/WDQ FC3 R': 'Unknown',
    'ANCG SYS LOUDSPE': 'MC System',
    '800 TON AC NO 5' : 'AC System',
    'VENT MACH SPCS O': 'Ventilation System',
    '800 TON AC PLANT': 'AC System',
    '800 TON AC NO 2': 'AC System',
    'VENT MACH SPCS I': 'Ventilation System',
    '800 TON AC NO 1' : 'AC System',
    'IC CKT 1MC MICRO': 'MC System',
    'Condenser': 'AC System',
    '800 TON AC NO 6': 'AC System',
    'CHW SYS AC PLT N' : 'Chilled Water System',
    '800 TON AC NO 3': 'AC System',
    '800 TON AC NO 4': 'AC System',
    'AC PLANT 4 SW CI': 'AC System',
    'AC PLANT 3 SW CI': 'AC System',
    'AC PLANT 6 SW CI': 'AC System',
    'CHW SYS MONITORI': 'Chilled Water System',
    'CHILLED WTR CLG': 'Chilled Water System',
    'AC PLANT 2 SW CI': 'AC System',
    'VENT MACH SPCS C': 'Ventilation System',
    'IC CKT 11VS AFS': 'MC System',
    'AC PLANT 5 SW CI': 'AC System',
    'AC PLANT 8 SW CI': 'AC System',
    'IC CKT 2MC AMPLI': 'MC System',
    'AC PLANT 7 SW CI': 'AC System',
    'AC PLANT 1 SW CI': 'AC System',
    'AC PLANT 3 COMPR': 'AC System',
    'VENT MACH SPCS U': 'Ventilation System',
    'IC CKT 17HF AIRF': 'MC System'  
}

jobs['equipment_type'] = jobs['equipment_nomenclature'].map(equipment_map)

In [62]:
# Only considering records with equipment_nomenclature in equipment_map
jobs_sub = jobs[(jobs['equipment_type'].notna()) & (jobs['equipment_type'] != 'Unknown')]
#jobs_sub['equipment_type'].value_counts().round(2). ~80% are Ventilation System

In [ ]:
#Ventilation, AC, MC, Heating, Chilled Water
#Variables to Consider: Deferral reason, component status, when discovered, action taken
#Binary: Supplies ordered
#Numeric: Total hours worked, days elapsed

# Probabilities of Requiring A Supply Order for Each Job Type

In [56]:
supply_prob = []
supply_prob.append(f"General Population : {jobs['supplies_ordered'].mean().round(2)}")

types = ['Ventilation System','AC System','MC System','Heating System', 'Chilled Water System']
for type in types:
    dat = jobs_sub[jobs_sub['equipment_type'] == type]
    supply_prob.append(f"{type} : {dat['supplies_ordered'].mean().round(2)}")
supply_prob



['General Population : 0.3',
 'Ventilation System : 0.29',
 'AC System : 0.37',
 'MC System : 0.27',
 'Heating System : 0.49',
 'Chilled Water System : 0.21']

# Deferral Reason Breakdown For Each Equipment Type

In [58]:
#Each of these categories is similar to 'Lack of facilities/capabilities'
to_group = [ "No formal training on this equipment","Not authorized for ship's force accomplishment","Inadequate school practical training",
       "Formal training inadequate for this equipment", "Lack of technical documentation"]
jobs['deferral_reason_reduced'] = jobs['deferral_reason'].apply(lambda x: 'Lack of facilities/capabilities' if x in to_group else x)

In [ ]:
def_reason_proportions = []
def_reason_proportions.append(f"General Population : {jobs['deferral_reason_reduced'].value_counts(normalize = True).round(2)}")

types = ['Ventilation System','AC System','MC System','Heating System', 'Chilled Water System']
for type in types:
    dat = jobs_sub[jobs_sub['equipment_type'] == type]
    def_reason_proportions.append(f"{type} : {dat['deferral_reason_reduced'].value_counts(normalize = True).round(2)}")
def_reason_proportions

<pre>
General Population : 
Lack of material                                         0.35
Due to ship's force work backlog/operational priority    0.24
Lack of facilities/capabilities                          0.17
Other - or not applicable                                0.14
For ship's force overhaul or availability work list      0.10


Ventilation System : 
Lack of material                                         0.34
Due to ship's force work backlog/operational priority    0.25
Other - or not applicable                                0.15
Lack of facilities/capabilities                          0.15
For ship's force overhaul or availability work list      0.11

AC System :
Lack of material                                         0.46
Lack of facilities/capabilities                          0.23
Due to ship's force work backlog/operational priority    0.14
Other - or not applicable                                0.10
For ship's force overhaul or availability work list      0.07


MC System : 
Lack of material                                         0.42
Due to ship's force work backlog/operational priority    0.31
For ship's force overhaul or availability work list      0.16
Lack of facilities/capabilities                          0.07
Other - or not applicable                                0.04


Heating System : 
Lack of material                                         0.49
Lack of facilities/capabilities                          0.16
Due to ship's force work backlog/operational priority    0.15
For ship's force overhaul or availability work list      0.12
Other - or not applicable                                0.07


Chilled Water System :
Lack of facilities/capabilities                          0.37
Lack of material                                         0.29
Due to ship's force work backlog/operational priority    0.19
Other - or not applicable                                0.09
For ship's force overhaul or availability work list      0.06

